In [1]:
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
from keras import backend as K
from keras_vggface import utils

#custom parameters
hidden_dim = 512
image_size = 224
train_dir = "dataset_train_chafg"
validation_dir = "dataset_valid_deep"
classes = 10
EPOCHS = 50

vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(classes, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
from keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(
      preprocessing_function=utils.preprocess_input,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(preprocessing_function=utils.preprocess_input)
 
# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 30
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 4343 images belonging to 10 classes.
Found 1363 images belonging to 10 classes.


In [3]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
custom_vgg_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = custom_vgg_model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/50
218/217 [==============================] - 114s 525ms/step - loss: 3.4903 - acc: 0.2197 - val_loss: 2.0121 - val_acc: 0.2480
Epoch 2/50
218/217 [==============================] - 106s 485ms/step - loss: 2.0115 - acc: 0.2440 - val_loss: 1.9898 - val_acc: 0.2524
Epoch 3/50
218/217 [==============================] - 106s 487ms/step - loss: 1.8792 - acc: 0.2888 - val_loss: 1.7128 - val_acc: 0.3191
Epoch 4/50
218/217 [==============================] - 106s 487ms/step - loss: 1.7283 - acc: 0.3277 - val_loss: 1.5826 - val_acc: 0.3690
Epoch 5/50
218/217 [==============================] - 106s 487ms/step - loss: 1.5822 - acc: 0.3619 - val_loss: 1.6179 - val_acc: 0.3690
Epoch 6/50
218/217 [==============================] - 106s 487ms/step - loss: 1.4963 - acc: 0.3925 - val_loss: 1.4065 - val_acc: 0.4197
Epoch 7/50
218/217 [==============================] - 106s 488ms/step - loss: 1.3930 - acc: 0.4207 - val_loss: 1.3675 - val_acc: 0.4358
Epoch 8/50
218/217 [============================

In [4]:
import matplotlib.pyplot as plt
import numpy as np

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("VGGFace50chafg")

In [5]:
import csv
import os
import cv2
from keras.preprocessing.image import img_to_array

def readFaces(dir,mydict):
    faces = []
    ages = []
    directory = os.fsencode(dir)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".png") or filename.endswith(".jpg"): 
            path = ".\\" + dir + "\\" + filename
            img = cv2.imread(path)
            img = cv2.resize(img, (224, 224))
            img = img.astype("float16")
            img = img_to_array(img)
            img = utils.preprocess_input(x=img,version=1)
            img = np.expand_dims(img, axis=0)
            faces.append(img)
            ages.append(mydict.get(filename))
    return np.asarray(faces),ages

with open('ground_truth/test_gt.csv', 'r') as f:
    reader = csv.reader(f)
    mydict = dict((rows[0],rows[1]) for rows in reader)

x_test,y_test = readFaces("test_deep",mydict)

def arrangeLabels(Y,bins,labels):
    returnLabels = []
    for label in Y:
        for index in range(len(bins)):
            if float(label) > bins[index]:
                continue;
            else:
                returnLabels.append(labels[index-1])
                break;
                
    return returnLabels

bins = [0.0, 6.0, 13.0, 20.0, 27.0, 34.0, 41.0, 48.0, 55.0, 62.0,np.inf]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

y_test = np.array(arrangeLabels(y_test,bins,labels))

hit = 0
total = len(x_test)
for index in range(total):
    # classify the input image
    print("[INFO] classifying image...")
    proba = custom_vgg_model.predict(x_test[index])[0]
    idx = np.argmax(proba)
    print("Predicted "+str(idx)," Actual "+str(y_test[index]))
    hit += idx==y_test[index]

print(hit/total)


[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 4  Actual 8
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 6  Actual 3
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 5  Actual 7
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 5  Actual 8
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 7  Actual 9
[INFO] classifying image...
Predicted 2  Actual 2
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 4  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 5


Predicted 6  Actual 6
[INFO] classifying image...
Predicted 6  Actual 8
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 6  Actual 5
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...


Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 7  Actual 9
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 6  Actual 5
[INFO] classifying image...
Predicted 1  Actual 2
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...


Predicted 3  Actual 4
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 4  Actual 7
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 2  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 7  Actual 7
[INFO] classifying image...
Predicted 3  Actual 1
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 7  Actual 7
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...


Predicted 3  Actual 5
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 5  Actual 7
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 7  Actual 8
[INFO] classifying image...
Predicted 7  Actual 8
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 2  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 5  Actual 9
[INFO] classifying image...


...

Predicted 2  Actual 2
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 7  Actual 9
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 7  Actual 6
[INFO] classifying image...
Predicted 2  Actual 1
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...


Predicted 6  Actual 5
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 2  Actual 0
[INFO] classifying image...
Predicted 2  Actual 1
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 2  Actual 1
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 6  Actual 5
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...


Predicted 4  Actual 5
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 3  Actual 5
[INFO] classifying image...
Predicted 5  Actual 7
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 6  Actual 8
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 6  Actual 5
[INFO] classifying image...
Predicted 2  Actual 2
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...


Predicted 3  Actual 4
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 2  Actual 2
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 7  Actual 7
[INFO] classifying image...
Predicted 7  Actual 7
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...


Predicted 7  Actual 4
[INFO] classifying image...
Predicted 5  Actual 4
0.4436695278969957


In [6]:
custom_vgg_model.save("VGGFace50chafg")